In [ ]:
!nvidia-smi

Sat Feb 27 17:59:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   1232MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers datasets

     |████████████████████████████████| 1.9MB 17.0MB/s 
     |████████████████████████████████| 184kB 48.4MB/s 
     |████████████████████████████████| 890kB 50.8MB/s 
     |████████████████████████████████| 3.2MB 51.3MB/s 
     |████████████████████████████████| 112kB 57.5MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=560a02df248f0a4e25142cb6dd8e5616a2edefc04b1bde060b734828e9069e1f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW
import torch.nn as nn
import torch
from datasets import load_dataset
from pytorchtools import EarlyStopping

In [ ]:
DOMAIN = 
BATCH_SIZE = 32
num_labels = 
n_epochs = 10
TASK = 

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

In [ ]:
config =  AutoConfig.from_pretrained('distilbert-base-uncased', num_labels=3)  # pass the num_classes here
pretrained_model = AutoModel.from_pretrained('distilbert-base-uncased', config=config)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', usefast=True, use_lower_case=True)

In [ ]:
class AutoModelPooled(nn.Module):
    def __init__(self, config, model):  # pass the config initialized to this model
        super().__init__()
        
        self.base_model = pretrained_model
        self.custom_head = nn.Sequential(
            nn.Linear(config.dim, config.dim),
            nn.ReLU(),
            nn.Dropout(config.seq_classif_dropout),
            nn.Linear(config.dim, config.num_labels),
        )

    def forward(self, input_ids, attention_mask=None):
        last_hidden_state = self.base_model(input_ids, attention_mask)[0]  # (bs, seq_len, dim)
        pooled_output = last_hidden_state[:, 0]  # (bs, dim)
        output = self.custom_head(pooled_output)
        
        return output

In [ ]:
model = AutoModelPooled(config, pretrained_model).to(device=device)

In [ ]:
dataset = load_dataset("multi_nli")

Reusing dataset multi_nli (/root/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451)


In [ ]:
#  filter according to genre
# start_with_ar = dataset.filter(lambda example: example['sentence1'].startswith('Ar'))
#{'government': 77350,
#  'telephone': 83348,
#  'fiction': 77348,
#  'travel': 77350,
#  'slate': 77306}

In [ ]:
encoded_dataset = dataset.map(lambda x: tokenizer(x['premise'], x['hypothesis'], padding='max_length', truncation=True), batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451/cache-f57bc9c117bf3c32.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451/cache-2c1186fd9c2fe3d4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451/cache-48b7b696d3e189da.arrow


In [ ]:
type(encoded_dataset['train'][0]['attention_mask'])

torch.Tensor

In [ ]:
encoded_dataset['train'] = encoded_dataset['train'].select(range(77302))

In [ ]:
train_data_loader = torch.utils.data.DataLoader(dataset = encoded_dataset['train'], batch_size=32, shuffle=True, num_workers=4)
val_data_loader = torch.utils.data.DataLoader(dataset = encoded_dataset['validation_matched'], batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# learning_rate = 1e-5
optimizer = AdamW(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train_model(model, patience, n_epochs):
    
    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = [] 
    valid_acc = []
    avg_valid_acc = [] 
    total = 0
    correct = 0
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=f"checkpoint_{DOMAIN}.pt")
    
    for epoch in range(1, n_epochs + 1):

        model.train() 
        for idx, d in enumerate(train_data_loader):
            input_ids = d['input_ids'].to(device=device)
            attention_mask = d['attention_mask'].to(device=device)
            target = d['label'].to(device=device)

            output = model(input_ids, attention_mask)

            loss = loss_fn(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        model.eval() 
        for idx, d in enumerate(val_data_loader):
            input_ids = d['input_ids'].to(device=device)
            attention_mask = d['attention_mask'].to(device=device)
            target = d['label'].to(device=device)

            output = model(input_ids, attention_mask)
            
            loss = loss_fn(output, target)
            valid_losses.append(loss.item())

            _, predicted = torch.max(output, dim=1)
            total += target.shape[0]
            correct += int((predicted==target).sum())

            valid_acc.append(correct/total)

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        valid_acc = np.average(valid_acc)
        
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        avg_valid_acc.append(valid_acc)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}'  +
                     f'valid_acc: {valid_acc:.5f}'  +)
        
        print(print_msg)
        
        train_losses = []
        valid_losses = []
        valid_acc = []
        total = 0
        correct = 0
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load(f"{task}-checkpoint_{DOMAIN}.pt"))

    return model, avg_train_losses, avg_valid_losses

In [ ]:
patience = 3

model, train_loss, valid_loss = train_model(model, patience, n_epochs)

In [1]:
output_results = {}

In [ ]:
encoded_dataset['train'] = encoded_dataset['train'].select(range(77302))  # filter dataset here to get different domain


In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()